In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('EV_files/EV_train.csv')
df.set_index('House ID', inplace=True)

houses_w_nans = df[pd.isnull(df).any(axis=1)]
houses_missing_data = houses_w_nans.index.tolist()

df = df[~df.index.isin(houses_missing_data)]

In [10]:
df.head(3)

,Interval_1,Interval_2,Interval_3,Interval_4,Interval_5,Interval_6,Interval_7,Interval_8,Interval_9,Interval_10,...,Interval_2871,Interval_2872,Interval_2873,Interval_2874,Interval_2875,Interval_2876,Interval_2877,Interval_2878,Interval_2879,Interval_2880
House ID,,,,,,,,,,,,,,,,,,,,,
11655099,0.950,0.826,0.361,0.238,0.342,0.233,0.351,0.194,0.292,0.234,...,0.664,0.783,0.601,0.639,0.417,0.439,0.226,0.190,0.710,0.728
11633257,0.353,0.327,0.358,0.292,0.285,0.304,0.361,0.342,0.355,0.348,...,0.536,0.558,0.622,0.634,0.513,0.421,0.273,0.296,0.291,0.289
11651552,0.150,0.181,0.150,0.150,0.131,0.125,0.088,0.106,0.094,1.019,...,2.125,0.881,0.481,1.194,0.138,0.119,0.038,0.088,0.056,0.113


In [11]:
df.shape

(1586, 2880)

Let's calculate the total demand for these 1586 homes and see if we can use a forecasting model to predict any weekly and half hourly periodicity in the data.

In [17]:
df.append(df.sum(), ignore_index=True)

,Interval_1,Interval_2,Interval_3,Interval_4,Interval_5,Interval_6,Interval_7,Interval_8,Interval_9,Interval_10,...,Interval_2871,Interval_2872,Interval_2873,Interval_2874,Interval_2875,Interval_2876,Interval_2877,Interval_2878,Interval_2879,Interval_2880
0,0.950000,0.826000,0.361000,0.238000,0.342,0.233000,0.351000,0.194000,0.292000,0.234000,...,0.664,0.783,0.601,0.639,0.417,0.439,0.226,0.190,0.710,0.728
1,0.353000,0.327000,0.358000,0.292000,0.285,0.304000,0.361000,0.342000,0.355000,0.348000,...,0.536,0.558,0.622,0.634,0.513,0.421,0.273,0.296,0.291,0.289
2,0.150000,0.181000,0.150000,0.150000,0.131,0.125000,0.088000,0.106000,0.094000,1.019000,...,2.125,0.881,0.481,1.194,0.138,0.119,0.038,0.088,0.056,0.113
3,2.088000,2.075000,2.121000,2.098000,2.046,2.081000,1.847000,0.420000,0.399000,0.364000,...,0.620,0.487,0.563,0.419,0.379,0.359,0.347,0.325,0.330,0.340
4,1.416000,1.250000,1.270000,1.258000,1.239,1.753105,4.609256,4.619256,4.075151,1.230000,...,1.596,1.667,1.569,1.664,1.580,1.635,1.568,1.565,1.575,1.571
5,1.124000,0.995000,0.792000,1.256000,0.996,1.203000,1.610000,1.562000,1.056000,1.374000,...,0.135,0.091,0.188,0.257,0.273,0.277,0.274,0.255,0.360,0.345
6,0.103000,0.126000,0.237000,0.209000,0.183,0.178000,0.198000,0.132000,0.114000,0.221000,...,0.564,0.516,0.564,0.487,0.481,0.390,0.518,0.746,0.255,0.192
7,0.685000,0.691000,0.701000,0.689000,0.712,0.693000,0.698000,0.709000,0.699000,0.724000,...,0.472,0.487,1.151,0.546,0.590,0.525,0.510,0.473,0.507,0.477
8,1.113000,1.098000,1.355000,1.513000,1.503,1.493000,1.490000,1.348000,1.475000,1.455000,...,4.053,3.113,1.413,0.780,1.013,1.043,0.768,1.038,1.000,0.750
9,0.433000,0.412000,0.378000,0.393000,0.399,0.400000,0.400000,0.342000,0.347000,0.850000,...,0.558,0.675,0.398,0.380,0.455,0.505,0.433,0.366,0.409,0.950


2880 30-min intervals span 60 days. Likely, not enough to capture seasonality but day of the week indicators or weekday/weekend dummy variables should certainly be applicable. Unfortunately, I don't have any dates associated with the variables to be apple to apply historical temperature data.